## install transformers

In [18]:
! pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.7 MB/s eta 0:00:00


In [20]:
! pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [1]:
! pip install transformers datasets evaluate seqeval


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 k

## Import libraries


In [1]:
from datasets import load_dataset, load_metric, ClassLabel, Sequence
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML

import transformers
from transformers import AutoTokenizer, BertForTokenClassification, TrainingArguments, Trainer
from transformers.models.bert.tokenization_bert_fast import BertTokenizerFast

from transformers import DataCollatorForTokenClassification




## Load the dataset

In [2]:
data = load_dataset('SpeedOfMagic/ontonotes_english')

  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 59924
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13900
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 8262
    })
})

In [4]:
data['train'][1]

{'tokens': ['Newsnight', 'returns', 'to', 'duo', 'action', 'tonight', '.'],
 'ner_tags': [31, 0, 0, 0, 0, 17, 0]}

## using NER task for token classification



In [5]:
task = "ner"
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [6]:
label = data['train'].features[f"{task}_tags"].feature.names
print(label)

['O', 'B-PERSON', 'I-PERSON', 'B-NORP', 'I-NORP', 'B-FAC', 'I-FAC', 'B-ORG', 'I-ORG', 'B-GPE', 'I-GPE', 'B-LOC', 'I-LOC', 'B-PRODUCT', 'I-PRODUCT', 'B-DATE', 'I-DATE', 'B-TIME', 'I-TIME', 'B-PERCENT', 'I-PERCENT', 'B-MONEY', 'I-MONEY', 'B-QUANTITY', 'I-QUANTITY', 'B-ORDINAL', 'I-ORDINAL', 'B-CARDINAL', 'I-CARDINAL', 'B-EVENT', 'I-EVENT', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'B-LAW', 'I-LAW', 'B-LANGUAGE', 'I-LANGUAGE']


## just to see how dataset look like

In [7]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(data['train'])

,tokens,ner_tags
0,"[Newport, Beach, telemarketers, concentrate, on, precious, metals, and, oil, -, leasing, deals, that, typically, cost, thousands, of, dollars, a, shot, .]","[B-GPE, I-GPE, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MONEY, I-MONEY, I-MONEY, O, O, O]"
1,"[I, can, imagine, .]","[O, O, O, O]"
2,"[She, could, have, four, or, five, years, ago, done, whatever, she, wanted, to, do, .]","[O, O, O, B-DATE, I-DATE, I-DATE, I-DATE, I-DATE, O, O, O, O, O, O, O]"
3,"[Slobodan, Milosevic, is, back, in, the, public, eye, .]","[B-PERSON, I-PERSON, O, O, O, O, O, O, O]"
4,"[I, do, n't, know, where, they, 're, based, .]","[O, O, O, O, O, O, O, O, O]"
5,"[Reporter, :]","[O, O]"
6,"[Jordeena, Ginsburg, from, News, 12, Westchester, has, the, story, .]","[B-PERSON, I-PERSON, O, B-ORG, I-ORG, I-ORG, O, O, O, O]"
7,"[like, I, can, read, a, book]","[O, O, O, O, O, O]"
8,"[They, think, it, 's, far, better, for, their, children, to, be, here, improving, their, English, than, sitting, at, home, in, front, of, a, TV, ., ""]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LANGUAGE, O, O, O, O, O, O, O, O, O, O, O]"
9,"[The, measure, now, goes, on, to, President, Clinton, who, said, he, will, sign, it, .]","[O, O, O, O, O, O, O, B-PERSON, O, O, O, O, O, O, O]"


## Lets preprocess the data

tokenize the data

In [9]:
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

In [10]:
# this will make sure the tokenizer is fast
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [11]:
## we need to preprocess the data since if our words already tokinze it will tokenize again wchich will end up giving more ids
## we will use truncation = true to reduce the size of the text that are bigger than maximum size and will use
## is_split_into_words=true then we align them
label_all_tokens = True # we can change the value of this flag to change the strategy to set the label for first token only
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], return_offsets_mapping=True, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [12]:
## use map to apply the above function on all sentences
tokenize_data = data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/13900 [00:00<?, ? examples/s]

In [13]:
tokenize_data['train'][0]

{'tokens': ['Sunday', 'the', 'interview', 'with', 'Bob', 'Shapiro', '.'],
 'ner_tags': [15, 0, 0, 0, 1, 2, 0],
 'input_ids': [101, 4465, 1996, 4357, 2007, 3960, 24630, 1012, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
 'offset_mapping': [[0, 0],
  [0, 6],
  [0, 3],
  [0, 9],
  [0, 4],
  [0, 3],
  [0, 7],
  [0, 1],
  [0, 0]],
 'labels': [-100, 15, 0, 0, 0, 1, 2, 0, -100]}

## fine tune the model

In [14]:
model = BertForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

## Initiate training
### before that we need training arguments which will need folder name and other optional

In [15]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

## make all our example to same size 

In [16]:
data_collector = DataCollatorForTokenClassification(tokenizer)

## last we need metric from prediction, we use seqeval metric

In [17]:
metric = load_metric("seqeval")

<ipython-input-17-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")



*   selecting predicted index for each token with max logit
*   cnvert it to string label
*   ignore everywhere we set label of -100 from earliear





In [18]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## accuracy/f1score/recall/precision

In [19]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenize_data["train"],
    eval_dataset=tokenize_data["validation"],
    data_collator=data_collector,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## Now we train our trainer

In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.109700,0.141701,0.827827,0.881092,0.853630,0.971826
2,0.078500,0.139428,0.835974,0.892198,0.863171,0.972847
3,0.060300,0.141873,0.836531,0.893985,0.864304,0.973009


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: -100 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarnin

TrainOutput(global_step=11238, training_loss=0.11477750186543381, metrics={'train_runtime': 2371.8928, 'train_samples_per_second': 75.793, 'train_steps_per_second': 4.738, 'total_flos': 5160448990795512.0, 'train_loss': 0.11477750186543381, 'epoch': 3.0})

In [23]:
trainer.evaluate()

{'eval_loss': 0.14187313616275787,
 'eval_precision': 0.8365310532744527,
 'eval_recall': 0.8939853398935637,
 'eval_f1': 0.8643044364624793,
 'eval_accuracy': 0.9730091761801127,
 'eval_runtime': 58.8352,
 'eval_samples_per_second': 236.253,
 'eval_steps_per_second': 14.77,
 'epoch': 3.0}

In [24]:
predictions, labels, _ = trainer.predict(tokenize_data["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'0': {'precision': 0.863961813842482,
  'recall': 0.8894348894348895,
  'f1': 0.8765133171912833,
  'number': 407},
 '1': {'precision': 0.8307210031347962,
  'recall': 0.8688524590163934,
  'f1': 0.8493589743589743,
  'number': 305},
 '100': {'precision': 0.8462043727127709,
  'recall': 0.8915765884524124,
  'f1': 0.8682981625611811,
  'number': 30344},
 '2': {'precision': 0.8379746835443038,
  'recall': 0.8619791666666666,
  'f1': 0.8498074454428755,
  'number': 384},
 '3': {'precision': 0.56,
  'recall': 0.6268656716417911,
  'f1': 0.591549295774648,
  'number': 67},
 '4': {'precision': 0.6910569105691057,
  'recall': 0.7264957264957265,
  'f1': 0.7083333333333334,
  'number': 117},
 '5': {'precision': 0.9052478134110787,
  'recall': 0.8871428571428571,
  'f1': 0.8961038961038962,
  'number': 700},
 '6': {'precision': 0.8624454148471615,
  'recall': 0.8643326039387309,
  'f1': 0.8633879781420765,
  'number': 457},
 '7': {'precision': 0.8337730870712401,
  'recall': 0.858695652173913